# Node.js 교과서 7장
# < MySQL >

mysql 설치 후 cmd에서 mysql을 실행하자
```
mysql -h localhost -u root -p
```

## 7.4 ~ 7.5 SQL문 작성은 아래를 참고
![](img/MySQL1.png)
![](img/MySQL2.png)

## 7.6 시퀄라이즈 사용하기
시퀄라이즈는 자바스크립트 객체와 데이터베이스 릴레이션을 매핑해주는 ORM(Object-Relational Mapping)도구이다.

In [1]:
%%writefile code/7/package.json
{
  "name": "learn-sequelize",
  "version": "0.0.1",
  "description": "시퀄라이즈를 배우자",
  "main": "app.js",
  "scripts": {
    "start": "nodemon app"
  },
  "author": "hyejin",
  "license": "MIT"
}

Writing code/7/package.json


**모듈 설치**
> npm i express morgan nunjucks sequelize sequelize-cli mysql2

 - sequelize-cli: 시퀄라이즈 명령어를 실행하기 위한 패키지 
 - mysql2: MySQL과 시퀄라이즈를 이어주는 드라이버
 
> npm i -D nodemon

> npx sequelize init

config, models, migations, seeders 폴더가 생성된다.  
models 폴더 안에 index.js가 있는데, sequelize-cli 설치 후 자동생성되는 코드가 들어가있다.  
그대로 사용하면 에러가 발생하고 필요없는 부분도 많으므로 아래와 같이 수정한다.

In [7]:
%%writefile code/7/models/index.js

const Sequelize = require('sequelize');

const env = process.env.NODE_ENV || 'development';
const config = require('../config/config')[env];
const db = {};

const sequelize = new Sequelize(config.database, config.username, config.password, config);

db.sequelize = sequelize;

module.exports = db;

Overwriting code/7/models/index.js


### 7.6.1 MySQL 연결하기

MySQL연동을 위해서는 config 폴더에 config.json 정보를 넣어주어야한다.  
MySQL 커넥션과 일치하도록 development.password와 development.database를 수정한다.

```json
{
  "development": {
    "username": "root",
    "password": "비밀번호",   < MySQL 접속 비밀번호를 넣는다.
    "database": "nodejs",     < 생성해둔 스키마 이름을 넣는다.
    "host": "127.0.0.1",
    "dialect": "mysql"
  },
  "test": {
    "username": "root",
    "password": null,
    "database": "database_test",
    "host": "127.0.0.1",
    "dialect": "mysql"
  },
  "production": {
    "username": "root",
    "password": null,
    "database": "database_production",
    "host": "127.0.0.1",
    "dialect": "mysql"
  }
}
```

In [8]:
%%writefile code/7/app.js

const express = require('express');
const path = require('path');
const morgan = require('morgan');
const nunjucks = require('nunjucks');

const { sequelize } = require('./models');

const app = express();
app.set('port', process.env.PORT || 3001);
app.set('view engine', 'html');
nunjucks.configure('views', {
  express: app,
  watch: true,
});
sequelize.sync({force:false})
  .then(() => {
    console.log('데이터베이스 연결 성공~');
  })
  .catch((err) => {
    console.log(err);
  });

app.use(morgan('dev'));
app.use(express.static(path.join(__dirname, 'public')));
app.use(express.json());
app.use(express.urlencoded({ extended: false }));

app.use((req, res, next) => {
  const error = new Error(`${req.method} ${req.url} 라우터가 없습니다.`);
  error.status = 404;
  next(error);
});

app.use((err, req, res, next) => {
  res.locals.message = err.message;
  res.locals.error = process.env.NODE_ENV !== 'production' ? err:{};
  res.status(err.status || 500);
  res.render('error');
});

app.listen(app.get('port'), () => {
  console.log(app.get('port'), ' 포트에서 대기중');
});


Overwriting code/7/app.js


> npm start  

```
3001  포트에서 대기중  
Executing (default): SELECT 1+1 AS result  
데이터베이스 연결 성공~
```
잘 연결된다

### 7.6.2 모델 정의하기
models에 user과 comment를 정의하자

In [16]:
%%writefile code/7/models/user.js
const Sequelize = require('sequelize');

module.exports = class User extends Sequelize.Model {
  static init(sequelize) {
    return super.init({
      // 시퀄라이즈에서는 primary key를 자동으로 생성해준다.
      name: {
        type: Sequelize.STRING(20),
        allowNull: false,
        unique: true,
      },
      age: {
        type: Sequelize.INTEGER.UNSIGNED,
        allowNull: false,
      },
      married: {
        type: Sequelize.BOOLEAN,
        allowNull: false,
      },
      comment: {
        type: Sequelize.TEXT,
        allowNull: true,
      },
      created_at: {
        type: Sequelize.DATE,
        allowNull: false,
        defaultValue: Sequelize.NOW,
      },
    }, {
      sequelize,
      timestamps: false, //timestamps가 true이면 createdAt(데이터생성시간), updatedAt(데이터수정시간)을 생성한다.
      underscored: false, //true이면, 테이블명과 컬럼명을 snake_case로 바꿈 (false일때는 camelCase)
      modelName: 'User', //노드에서 사용할 모델 이름
      tableName: 'users', //데이터베이스에서의 테이블 이름
      paranoid: false, //true이면 deletedAt에 삭제 시간을 기록한다. 삭제하지 않은 데이터의 deletedAt은 null이된다. (soft delete 처리)
      charset: 'utf8',
      collate: 'utf8_general_ci',
    });
  }
  static associate(db){}
};

Writing code/7/models/user.js


In [17]:
%%writefile code/7/models/comment.js

const Sequelize = require('sequelize');

module.exports = class Comment extends Sequelize.Model {
  static init(sequelize) {
    return super.init({
      comment: {
        type: Sequelize.STRING(100),
        allowNull: false,
      },
      created_at: {
        type: Sequelize.DATE,
        allowNull: true,
        defaultValue: Sequelize.NOW,
      },
    }, {
      sequelize,
      timestamps: false,
      modelName: 'Comment',
      tableName: 'comments',
      paranoid: false,
      charset: 'utf8mb4',
      collate: 'utf8mb4_general_ci',
    });
  }
  static associate(db){}
};

Writing code/7/models/comment.js


모델을 생성했다면 models/index.js와 연결한다.

In [20]:
%%writefile code/7/models/index.js

const Sequelize = require('sequelize');
const User = require('./user');
const Comment = require('./comment');

const env = process.env.NODE_ENV || 'development';
const config = require('../config/config')[env];
const db = {};

const sequelize = new Sequelize(config.database, config.username, config.password, config);

db.sequelize = sequelize;

db.User = User;
db.Comment = Comment;

User.init(sequelize);
Comment.init(sequelize);

User.associate(db);
Comment.associate(db);

module.exports = db;

Overwriting code/7/models/index.js


### 7.6.3 관계 정의하기
**관계**
#### 1. 1:N (일대다)
 - hasMany와 belongsTo 메서드로 표현한다.
 - users(사용자)는 여러개의 comment(댓글)을 남길 수 있다.
 - 댓글을 남긴 사용자는 commenter라고 하자.
 ```js
// models/user.js
    static associate(db){
        db.User.hasMany(db.Comment, {foreignKey: 'commenter', sourceKey: 'id'});
    } //user의 id를 comment가 commenter로 참조한다.
 ```
 - belongsTo 메서드: 다른 모델의 정보가 들어가는 테이블에 사용함
 ```js
// models/comment.js
   static associate(db){
       db.Comment.belongsTo(db.User, {foreignKey: 'commenter', targetKey: 'id'});
   } //comment가 user의 id를 commenter로 참조한다. 즉 commenter에 user 참조키가 생긴다.
 ```
 
#### 2. 1:1 (일대일)
 - hasOne과 belongsTo 메서드로 표현한다.
 - 사용자(user)마다 사용자 정보(info)가 있다.
 - 각 사용자를 UserId로 구별한다.
 ```js
    static associate(db){
        db.User.hasOne(db.Info, {foreignKey: 'UserId', sourceKey: 'id'});
    } 
 ```
 ```js
   static associate(db){
       db.Info.belongsTo(db.User, {foreignKey: 'UserId', targetKey: 'id'});
   } //info에 user 참조키가 생긴다.
 ```

#### 3. N:M (다대다)
 - ex) 게시글과 해시태그
 - 하나의 게시글이 여러개의 해시태그를 가질 수 있고, 하나의 해시태그가 여러개의 게시글을 가질 수 있음
 - DB 특성상 다대다 관계는 중간 테이블 필요 (시퀄라이저가 알아서 만들어준다)



![](img/nm.png)



 - belongsToMany 메서드로 표현한다.
 ```js
    static associate(db){
        db.Post.belongsToMany(db.HashTag, {through: 'PostHashTag'});
    } 
 ```
 ```js
   static associate(db){
       db.HashTag.belongsToMany(db.Post, {through: 'PostHashTag'});
   }
 ```
 - 자동으로 생성된 PostHashTag 모델에는 다음과 같이 접근할 수 있다.
 ```js
    db.sequelize.models.PostHashTag
 ```

----------
### 7.6.4 쿼리 알아보기
쿼리는 프로미스를 반환하므로 then을 붙여 결과값을 받을 수 있다.
#### 1. INSERT
```SQL
INSERT INTO nodejs.users(name, age, married, comment)
VALUES('zero',24,0,'자기소개1');
```
➡
```js
const {User} = require('../models');
User.create({
    name: zero,
    age: 24,
    married: false,
    comment: '자기소개1',
});
```

#### 2. SELECT
```SQL
SELECT * FROM nodejs.users;
```
➡
```js
User.findAll({});
```
-----
```SQL
SELECT * FROM nodejs.users LIMIT 1;
```
➡
```js
User.findOne({});
```
---

```SQL
SELECT name, married FROM nodejs.users;
```
➡
```js
User.findAll({
    attributes:['name', 'married'],
});
```
---

- Op.gt (초과) >
- Op.gte (이상) >=
- Op.lt (미만) <
- Op.lte (이하) <=
- Op.ne (같지 않음) !=
- Op.or (또는)
- Op.in (배열 요소 중 하나)
- Op.notIn (배열 요소와 모두 다름)


```SQL
SELECT name, age FROM nodejs.users WHERE married=1 AND age>30;
```
➡
```js
User.findAll({
    attributes:['name', 'age'],
    where: {
        married: ture,
        age:{[Op.gt]:30},
    },
});
```


---
```SQL
SELECT name, age FROM nodejs.users WHERE married=1 OR age>30;
```
➡
```js
User.findAll({
    attributes:['name', 'age'],
    where: {
        [Op.or]: [{married: ture}, {age:{[Op.gt]:30}}],
        
    },
});
```


---

#### 3. UPDATE
```SQL
UPDATE nodejs.users SET commnet = '수정내용' WHERE id = 2;
```
➡
```js
User.update({
    comment: '수정내용',
},{
    where: {id: 2},
});
```

#### 4. DELETE
```SQL
DELETE FROM nodejs.users WHERE id = 2;
```
➡
```js
User.destory({
    where: {id: 2},
});
```
---
```SQL
DELETE FROM nodejs.users WHERE id = 1 OR id = 3 OR id = 5;
```
➡
```js
User.destory({
    where: {id: { [Op.in]: [1,3,5] } },
});
```

---
#### 7.6.4.1 관계 쿼리
- 결과값이 자바스크립트 객체
```js
const user = await User.findOne({});
console.log(user.nick); //사용자 닉네임 출력
```
(findAll은 모델의 배열을 반환함)



- include로 JOIN과 비슷한 기능 수행 가능(관계 있는것 엮기)  
include 속성으로 특정 사용자와 관련된 댓글을 모두 가져온다  

```js
const user = await User.findOne({
    include:[{
        model: Comment,
    }]
});

console.log(user.Comments);
```

- 다대다 모델은 다음과 같이 접근 가능
  
```js
db.sequelize.models.PostHashing
```

- get + 모델명으로 관계있는 데이터 로딩 가능
```js
const user = await User.findOne({});
const comments = await user.getComments();
console.log(comments); //사용자 댓글
```

- as로 모델명 변경 가능
```js
db.User.hasMany(db.Comment, { foreignKey: 'commenter', sourceKey: 'id', as: 'Answers'});
```

- 관계 추가
```js
const user = await User.findOne({});
const comment = await Comment.create();
await user.addComment(comment);
// 또는
await user.addComment(comment.id);
```

- 여러 개 관계를 배열로 추가하기
```js
const user = await User.findOne({});
const comment1 = await Comment.create();
const comment2 = await Comment.create();
await user.addComment([comment1, comment2]);
```
----
 - 추가 add+모델명
 - 찾기 get+모델명
 - 삭제 remove+모델명
 - 수정 set+모델명


#### 7.6.4.2 raw 쿼리
시퀄라이즈 쿼리를 사용하기 싫다면... SQL문을 직접 쓸 수도 있다.

```js
    const [result, metadata] = await sequelize.query('SELECT * FROM comments');
    console.log(result);

```